<a href="https://colab.research.google.com/github/JuliethLopez/chatbot_simpsons/blob/master/Pre_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# librerias
import pickle
import re, string
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer

# Cargar modelo y preprocesamiento

In [129]:
!unzip '/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip' #julieth

Archive:  /content/drive/My Drive/Simpsons Chat bot/Modelos guardos/clasificador.zip
replace clasificador/variables/variables.data-00000-of-00002? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [130]:
modelo_clasificador = tf.keras.models.load_model('clasificador')
#Check its architecture
#modelo_clasificador.summary()

### Cargar para preprocesamiento
Cargamos el label_tokenizer, el encodeer, los trigramas y bigramas.

In [132]:
#label tokenizer
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/label_tokenizer.pickle', 'rb') as f:
  label_tokenizer = pickle.load(f)

In [133]:
#encoder
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/encoder.pickle', 'rb') as f:
  encoder = pickle.load(f)

In [134]:
#trigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqnew.pickle', 'rb') as f:
  freqnew = pickle.load(f)

In [135]:
#bigramas
with open('/content/drive/My Drive/Simpsons Chat bot/Modelos guardos/freqbnew.pickle', 'rb') as f:
  freqbnew = pickle.load(f)

### Función clasificadora

In [136]:
# funcion de frase tokenizada a palabras
reverse_word_map = dict(map(reversed, label_tokenizer.word_index.items()))

def sequence_to_text(list_of_indices):
    # Busca palabras en el diccionario
    label = [reverse_word_map.get(i) for i in list_of_indices]
    return(label)

In [137]:
def clasifica (sentence):
  sentence = re.sub(' +', ' ',sentence.lower()) #sentence en minusculas
  sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence) #quita signos de puntuación
  # preprocesamiento trigramas
  vectorizer3 = CountVectorizer(vocabulary=freqnew.keys(), ngram_range=(3,3))
  X3 = vectorizer3.fit_transform([sentence])
  #sequences = X3.toarray()
  # preprocesamiento bigramas
  vectorizer2 = CountVectorizer(vocabulary=freqbnew.keys(), ngram_range=(2,2))
  X2 = vectorizer2.fit_transform([sentence])
  #sequences_b = X2.toarray()
  # concatenamos
  sequences_bt = np.concatenate((X3.toarray(), X2.toarray()), axis=1)
  # predicción
  pred = modelo_clasificador.predict_classes(sequences_bt) #prediccion del modelo
  pred = encoder.inverse_transform(pred) #de label encoder a tokenizer
  pred = sequence_to_text(pred) #de tokenizer a label
  return(pred)

# Uso del clasificador

In [138]:
# frase del usuario
sentence = "Would this be a good time to be honest?"
clasifica(sentence)

['homer_simpson']